In [25]:
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pickle
import mlflow
from sklearn.metrics import root_mean_squared_error
import numpy as np

In [26]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops/experiment_tracking/mlruns/1', creation_time=1746639079004, experiment_id='1', last_update_time=1746639079004, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [27]:
df = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [28]:
# df.info()

In [29]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)
bf_outliers = df.shape[0]
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [30]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df.shape

(3009173, 20)

In [31]:
fraction = df.shape[0]/bf_outliers
fraction

0.9812202822125979

In [32]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

In [33]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


df_train = read_dataframe('data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2023-02.parquet')

In [34]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [35]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [36]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Anderson")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

In [37]:
root_mean_squared_error(y_train, lr.predict(X_train))

8.599015540104038

In [38]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [39]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [40]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("developer", "Anderson")
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        # rmse = mean_squared_error(y_val, y_pred, squared=False)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [41]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 20, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=2,
    trials=Trials()
)

  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

/home/andersonc/anaconda3/envs/ml-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:18:39] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.38641                          
[1]	validation-rmse:7.23728                          
[2]	validation-rmse:6.47264                          
[3]	validation-rmse:5.97829                          
[4]	validation-rmse:5.66433                          
[5]	validation-rmse:5.46594                          
[6]	validation-rmse:5.33837                          
[7]	validation-rmse:5.25847                          
[8]	validation-rmse:5.20427                          
[9]	validation-rmse:5.16416                          
[10]	validation-rmse:5.13775                         
[11]	validation-rmse:5.11688                         
[12]	validation-rmse:5.10005                         
[13]	validation-rmse:5.08650                         
[14]	validation-rmse:5.07655                         
[15]	validation-rmse:5.06775                         
[16]	validation-rmse:5.06026                         
[17]	validation-rmse:5.05509                         
[18]	validation-rmse:5.05031

/home/andersonc/anaconda3/envs/ml-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:21:53] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.59529                                                    
[1]	validation-rmse:6.22085                                                    
[2]	validation-rmse:5.50604                                                    
[3]	validation-rmse:5.14461                                                    
[4]	validation-rmse:4.96201                                                    
[5]	validation-rmse:4.85915                                                    
[6]	validation-rmse:4.80479                                                    
[7]	validation-rmse:4.77314                                                    
[8]	validation-rmse:4.75173                                                    
[9]	validation-rmse:4.73638                                                    
[10]	validation-rmse:4.72642                                                   
[11]	validation-rmse:4.71747                                                   
[12]	validation-rmse:4.70813            

In [42]:
mlflow.xgboost.autolog(disable=True)

In [45]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/andersonc/anaconda3/envs/ml-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:53:24] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.34798
[1]	validation-rmse:8.71461
[2]	validation-rmse:8.15750
[3]	validation-rmse:7.66878
[4]	validation-rmse:7.24278
[5]	validation-rmse:6.87279
[6]	validation-rmse:6.55280
[7]	validation-rmse:6.27351
[8]	validation-rmse:6.03615
[9]	validation-rmse:5.83215
[10]	validation-rmse:5.65607
[11]	validation-rmse:5.50742
[12]	validation-rmse:5.38152
[13]	validation-rmse:5.27467
[14]	validation-rmse:5.18091
[15]	validation-rmse:5.10423
[16]	validation-rmse:5.03863
[17]	validation-rmse:4.98333
[18]	validation-rmse:4.93645
[19]	validation-rmse:4.89665
[20]	validation-rmse:4.86284
[21]	validation-rmse:4.83233
[22]	validation-rmse:4.80801
[23]	validation-rmse:4.78633
[24]	validation-rmse:4.76663
[25]	validation-rmse:4.75023
[26]	validation-rmse:4.73620
[27]	validation-rmse:4.72441
[28]	validation-rmse:4.71360
[29]	validation-rmse:4.70176
[30]	validation-rmse:4.69350
[31]	validation-rmse:4.68619
[32]	validation-rmse:4.67969
[33]	validation-rmse:4.67432
[34]	validation-rmse:4.6

/home/andersonc/anaconda3/envs/ml-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


PermissionError: [Errno 13] Permission denied: '/workspaces'

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        